In [102]:
import autogen
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv(dotenv_path=".env")

llm_config = {"model": "gpt-4o-mini"}

# ANSI escape code for green text
GREEN = "\033[92;1m"
BLUE_BOLD = "\033[94;1m"
RESET = "\033[0m"

In [103]:
# Customer Proxy Agent
customer_proxy_agent = autogen.ConversableAgent(
    name="Customer_Proxy",
    system_message="""You represent the customer and are responsible for clearly communicating the customer’s requirements to the Product Owner. 
    Provide a high-level overview of the project’s goals, desired features, and key expectations for the end product. Focus on the impact and outcomes 
    that the customer wants to achieve, rather than technical details.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Product Owner Agent
product_owner_agent = autogen.ConversableAgent(
    name="Product_Owner",
    system_message="""As the Product Owner, you own the product vision and define the high-level requirements. Upon receiving customer requirements:
    - Translate these requirements into high-level product goals and key performance indicators (KPIs).
    - Break down the vision into prioritized epics and user stories, identifying features that will deliver maximum value.
    - Work closely with the Scrum Master to ensure these goals are achievable and establish a roadmap, starting with an initial backlog.
    - Your aim is to clearly define what the product should achieve and organize work into user stories that reflect this vision.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Scrum Master Agent
scrum_master_agent = autogen.ConversableAgent(
    name="Scrum_Master",
    system_message="""As the Scrum Master, you facilitate the Scrum process and support the team in following Scrum principles. Upon receiving the high-level 
    product goals, user stories, and initial backlog from the Product Owner:
    - Work with the team to clarify requirements and identify dependencies.
    - Outline the steps for initial sprint planning, estimating timeframes, assigning responsibilities, and setting sprint goals.
    - Provide guidance to the Project Manager, giving a clear structure to prioritize work, align sprints with high-level goals, and manage the project timeline.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Project Manager Agent
project_manager_agent = autogen.ConversableAgent(
    name="Project_Manager",
    system_message="""As the Project Manager, you are responsible for creating a detailed project plan, ensuring all components align with the 
    product goals and team capacity. After receiving the initial sprint structure and backlog from the Scrum Master:
    - Coordinate with each engineering team (Requirements, System, Software, Test, and Documentation) to gather estimates for user stories.
    - Use these estimates to structure a timeline, identifying milestones and dependencies.
    - Compile these responses to provide the Scrum Master with an organized backlog that outlines work estimates, planned sprints, and overall 
    project duration, ensuring the team can track progress effectively.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Requirement Engineer Agent
requirement_engineer_agent = autogen.ConversableAgent(
    name="Requirement_Engineer",
    system_message="""As the Requirement Engineer, you create detailed user stories and acceptance criteria that clarify what the end product 
    should accomplish. First Upon receiving the initial product goals and backlog from the Project Manager:
    - Convert high-level requirements into structured, detailed user stories, each with unique identifiers, descriptions, and acceptance criteria.
    - Estimate the effort required for each user story, considering complexity and historical productivity data.
    - Organize the user stories into logical groups or sprints, making it easier for the team to understand dependencies and priorities.
    - Report these user stories and estimates back to the Project Manager, providing a high-level summary of your breakdown and key requirements.
    
    Then calculate the {effort} to create the requirements document in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of requirements for the use-cases needed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many requirements can be completed every day} 
                            - step 3. effort = {work}/{productivity} .

    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# System Engineer Agent
system_engineer_agent = autogen.ConversableAgent(
    name="System_Engineer",
    system_message="""As the System Engineer, you design the technical structure and architecture of the product. After receiving detailed 
    user stories from the Requirement Engineer:
    - Develop a system design document, outlining architecture components, data flow, and integration points.
    - Create a data model that aligns with the system design, ensuring it supports the planned features and requirements.
    - Estimate the effort required to complete each section of the system design, using past productivity data to validate estimates.
    - Provide the Project Manager with a structured report of the system design, highlighting key components, dependencies, and overall effort estimates.
    
    Then, calculate the {effort} to create the design document in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of pages in the design document for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many pages can be completed every day} 
                            - step 3. effort = {work}/{productivity} .

    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Software Engineer Agent
software_engineer_agent = autogen.ConversableAgent(
    name="Software_Engineer",
    system_message="""As the Software Engineer, you implement the technical requirements and code the product features. After receiving the 
    system design from the System Engineer:
    - Break down the design into coding tasks, estimating lines of code (SLOC) for each major task.
    - Use past productivity data to estimate time required for development, unit testing, and code review.
    - Consider additional factors like refactoring, peer review, and integration testing in your estimates.
    - Report your coding tasks, total estimated SLOC, and effort requirements back to the Project Manager, focusing on feature-level progress.
    
    Then, calculate the {effort} to write the source code (SLOC) in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of lines of code (SLOC) that will be created for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many SLOC can be completed every day} 
                            - step 3. effort = {work}/{productivity} .

    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Test Engineer Agent
test_engineer_agent = autogen.ConversableAgent(
    name="Test_Engineer",
    system_message="""As the Test Engineer, you ensure that the product meets quality and functionality standards. After receiving the coding 
    tasks from the Software Engineer:
    - Develop test cases for each feature, based on the acceptance criteria from user stories.
    - Estimate time for test case creation, execution, and defect tracking, using historical productivity data to ensure accuracy.
    - Summarize the test plan, including estimated total test cases, effort, and critical tasks for defect resolution and retesting.
    - Provide this structured test plan to the Project Manager to inform sprint planning and backlog prioritization.
    
    Then, calculate the {effort} to write and execute the test cases in the test plan in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of test cases that will be created adn executed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many test cases can be completed every day} 
                            - step 3. effort = {work}/{productivity} .
    
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Documentation Engineer Agent
documentation_engineer_agent = autogen.ConversableAgent(
    name="Documentation_Engineer",
    system_message="""As the Documentation Engineer, you create user and training documentation to support the final product. After receiving 
    the completed features and test cases from the Test Engineer:
    - Draft user guides, technical documentation, and training materials based on feature specifications and test outcomes.
    - Estimate time for documentation tasks, factoring in reviews and any rework requirements.
    - Report your documentation tasks, total estimated pages, and effort requirements back to the Project Manager, providing a summary of 
    each document and its alignment with the project’s overall goals.
    
    Then, calculate the {effort} to write documentation pages in the documentation adn traingin phase in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of documentation pages that will be created adn executed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many pages can be completed every day} 
                            - step 3. effort = {work}/{productivity} .
    
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

[autogen.oai.client: 11-07 14:59:23] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 14:59:23] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 14:59:23] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 14:59:23] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 14:59:23] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 14:59:23] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 14:59:23] {164} WARNING - The API key specified is not a valid OpenAI format; i

In [104]:
# Initial conversation: Customer Proxy to Product Owner
customer_message = (
    "I want to create an AI-powered computer vision system that detects anomalies in the skin, such as cancer and psoriasis. "
    "The system should process high-resolution images, provide real-time diagnostics, and support doctors with detailed reports. "
    "The priorities are accuracy, speed, and usability in a healthcare setting."
)

# Product Owner’s prompt to Scrum Master
product_owner_to_scrum_master_prompt = (
    "I have gathered the requirements from the Customer Proxy"
    "Please outline the key tasks, structure the initial project timeline, and coordinate with the Project Manager to create a comprehensive project plan. "
    "This should include detailed user stories, a prioritized product backlog, and effort estimates for each engineering phase, aligned with our high-level goals."
)

# Scrum Master’s prompt to Project Manager
scrum_master_to_project_manager_prompt = (
    "The Product Owner has shared the project vision and an initial backlog, including prioritized user stories. "
    "Coordinate with each engineering team to refine and expand the user stories, creating a detailed product backlog and effort estimates for each phase. "
    "Ensure that each user story is complete with acceptance criteria, and compile a structured report with these details for our initial sprint planning."
)

# Project Manager’s prompt to Requirement Engineer
project_manager_to_requirement_engineer_prompt = (
    "The Scrum Master has outlined the initial sprint planning requirements based on the Product Owner’s input. "
    "As the Requirements Engineer, please expand upon these requirements by creating detailed user stories, complete with acceptance criteria and effort estimates. "
    "Use the provided waterfall-based historical data to estimate the effort required for this Agile project. Adapt this data by applying software project management principles, "
    "such as using story points and relative sizing, to determine the effort and productivity rates in an Agile context. "
    "Provide a structured report summarizing the user stories, estimates, and effort calculations specific to the requirement engineering phase to support sprint planning. "
    "At the end, provide a summary of the total number of user stories, requirements, and the overall effort for this phase."

    "Here is the historical data, based on a waterfall approach, for reference as you adapt it for Agile planning:"
    
    # Provide historical project data here
    "Project 1"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Requirement                           112 Req                 5 Req/Hour"
    "Review Requirments                                                            "
    "       Preparation for Review                112  Req               18 Req/Hour"
    "       Review Meeting                        112 Req                28 Req/Hour"
    "Rework                                       136 defects            10 defects/Hour"

    "Project 2"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Requirement                           389 Req                 3 Req/Hour"
    "Review Requirments                                                      "
    "       Preparation for Review                389  Req               5 Req/Hour"
    "       Review Meeting                        389 Req                8 Req/Hour"
    "Rework                                       413 defects            5 defects/Hour"

    "Project 3"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Requirement                           210 Req                 4 Req/Hour"
    "Review Requirments                                                      "
    "       Preparation for Review                210  Req               12 Req/Hour"
    "       Review Meeting                        210 Req                18 Req/Hour"
    "Rework                                       239 defects            8 defects/Hour"

    "Once completed, report back to me so I can proceed with the next steps in the project plan."
)

# Project Manager’s prompt to System Engineer
project_manager_to_system_engineer_prompt = (
    "The Requirements Engineer has completed detailed user stories. Based on this, please draft user stories for system design and data model creation, "
    "providing effort estimates for each task. Use the historical productivity data from the waterfall model as a relative guide and apply Agile estimation principles to derive "
    "effort in terms of story points and sprints. Summarize the user stories, estimates, and calculations for the system engineering phase. "
    "Provide a structured report with the total number of user stories, requirements, and overall effort."

    "Here is the waterfall-based historical data, to be used as a reference in estimating Agile-based effort:"
    
    # Provide historical project data here

    "Project 1"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Analysis Document                      66 Pages                 5 Pages/Hour"
    "Review AD                                                            "
    "       Preparation for AD                 66 Pages                    4 Pages/Hour"
    "       Review Meeting                        66 Pages                9 Pages/Hour"
    "Rework                                       88 defects            6 defects/Hour"
    "Write Design Document                       123 defects                 6 Page/Hour"
    "Review DD                                                            "
    "       Preparation for DD                 89 Pages              4 Req/Hour"
    "       Review Meeting                        89 Pages                8 Pages/Hour"
    "Rework                                       188 defects            5 defects/Hour"
    "Write Data Model (DM)                        28 Pages               1page/4hours"
    "Review DM                                                                     "
    "       Preparation for DM                 28 Pages                   4 page/hour"
    "       Review Meeting                        28 Pages                5 page/hour"
    "Rework                                       88 defects            6 defects/Hour"

    "Project 2"
    "Task                                                          Amount of Work      Productivity Rate"
    "Write Analysis/Design Document                                     234 Pages                 4 Pages/Hour"
    "Review Analysis/Design Document                                                           "
    "       Preparation for Analysis/Design Document                   234 Pages              5 Pages/Hour"
    "       Review Meeting                                             234 Pages               8 Pages/Hour"
    "Rework                                                             509 defects            7 defects/Hour"
    "Write Data Model (DM)                                             62 Pages               1 page/ hour"
    "Review DM                                                                     "
    "       Preparation for DM                    62 Pages                 4 page/hour"
    "       Review Meeting                        62 Pages                 10 page/hour"
    "Rework                                       378 defects            6 defects/Hour"

    "Project 3"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Analysis Document                      172 Pages                 5 Pages/Hour"
    "Review AD                                                            "
    "       Preparation for AD                   172 Pages                    5 Pages/Hour"
    "       Review Meeting                        172 Pages                9 Pages/Hour"
    "Rework                                       347 defects            7 defects/Hour"
    "Write Design Document                       220 Pages                 5 Pages/Hour"
    "Review DD                                                            "
    "       Preparation for DD                    220 Pages              5 Pages/Hour"
    "       Review Meeting                        220 Pages               8 Pages/Hour"
    "Rework                                       472 defects            6 defects/Hour"

    "Once completed, report back to me so I can proceed with the next steps in the project plan."
)

# Project Manager’s prompt to Software Engineer
project_manager_to_software_engineer_prompt = (
    "The System Engineer has finalized the design phase user stories. Based on the system design, please create user stories for coding tasks, estimating the required effort for each. "
    "To determine these estimates, use the historical waterfall-based data as a reference and adapt it for Agile estimation using story points, relative sizing, and sprint-based effort. "
    "Provide a structured report that summarizes user stories, effort estimates, and calculations for the software engineering phase, supporting the overall sprint planning. "
    "Ensure your report includes the total number of user stories, requirements, and overall effort."

    "Here is the historical waterfall data, to serve as a reference in calculating Agile-based estimates:"
    
    "Project 1"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Code                                  3800 SLOC                 5 SLOC/Hour"
    "Unit Testing                                                                     "
    "       Prepare/Execute Test Cases            256 test cases           10 Test Cases/Day"
    "       Fix Found Defects                     179 Defects              8 Defects/Day"
    "       Test Fixed Defects                    179 Defects              12 Defects/Day"
    "Code Inspection                                                                   "
    "       Preparation for Code Inspection       3800 SLOC                 113 SLOC/Hour"
    "       Code Inspection Meeting               3800 SLOC                 189 SLOC/Hour"
    "Rework                                       188 defects                5 defects/Hour"

    "Project 2"
    "Task                                    Amount of Work           Productivity Rate"
    "Write Code                                  9725 SLOC                 5 SLOC/Hour"
    "Unit Testing                                                                     "
    "       Prepare/Execute Test Cases            1045 test cases           5 Test Cases/Day"
    "       Fix Found Defects                     902 Defects              12 Defects/Day"
    "       Test Fixed Defects                    902 Defects              8 Defects/Day"
    "Code Inspection                                                                   "
    "       Preparation for Code Inspection       9725 SLOC                 90 SLOC/Hour"
    "       Code Inspection Meeting               9725 SLOC                 135 SLOC/Hour"
    "Rework                                       1230 defects                5 defects/Hour"

    "Project 3"
    "Task                                    Amount of Work           Productivity Rate"
    "Write Code                                  3800 SLOC                 5 SLOC/Hour"
    "Unit Testing                                                                     "
    "       Prepare/Execute Test Cases            332 test cases           25 Test Cases/Day"
    "       Fix Found Defects                     266 Defects              10 Defects/Day"
    "       Test Fixed Defects                    266 Defects              10 Defects/Day"
    "Code Inspection                                                                   "
    "       Preparation for Code Inspection       3800 SLOC                 102 SLOC/Hour"
    "       Code Inspection Meeting               3800 SLOC                 102 SLOC/Hour"
    "Rework                                       334 defects                5 defects/Hour"

    "Once completed, report back to me so I can proceed with the next steps in the project plan."
)

# Project Manager’s prompt to Test Engineer
project_manager_to_test_engineer_prompt = (
    "The Software Engineer has outlined user stories for coding tasks. Based on this, please prepare user stories for test planning and execution, "
    "including effort estimates for test case creation, execution, and rework. Reference the waterfall-based historical data, but adapt it for Agile by using story points, "
    "sprint estimates, and relative complexity to calculate the overall effort. Provide a structured report summarizing user stories, effort estimates, and calculations for the testing phase. "
    "Include in your summary the total number of user stories, test cases, and overall effort."

    "Here is the historical data in waterfall terms; use it to guide your Agile-based estimation:"
    
    
    "Project 1"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Test Plan (TP)                    166 pages            5 Pages/Day"
    "Review TP                                                            "
    "       Preparation for Review            166 Pages           4 Pages/Hour"
    "       Review Meeting                    166 Pages            8 Pages/Hour"
    "Rework                                   255 defects          5 defects/Hour"
    "Execute Test Plan                        215 Test Cases     9Test Cases/day"
    "Fix Found Defects                        213 Defects          14 Defects/day"
    "Test Fixed Defects                       213 Defects          17 Defects/day"

    "Project 2"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Test Plan (TP)                    417 pages            8 Pages/Day"
    "Review TP                                                            "
    "       Preparation for Review            417 Pages           5 Pages/Hour"
    "       Review Meeting                    417 Pages            10 Pages/Hour"
    "Rework                                   502 defects          5 defects/Hour"
    "Execute Test Plan                        1167 Test Cases      8 Test Cases/day"
    "Fix Found Defects                        645 Defects          5 Defects/day"

    "Project 3"
    "Task                                    Amount of Work      Productivity Rate"
    "Write Test Plan (TP)                    205 pages            8 Pages/Day"
    "Review TP                                                            "
    "       Preparation for Review            205 Pages           5 Pages/Hour"
    "       Review Meeting                    205 Pages            10 Pages/Hour"
    "Rework                                   101 defects          4 defects/Hour"
    "Execute Test Plan                        194 Test Cases      8 Test Cases/day"
    "Fix Found Defects                        113 Defects          5 Defects/day"
    "Test Fixed Defect                        113 Defects          10 Defects/day"

    "Once completed, report back to me so we can proceed with the next steps in the project plan."
)

# Project Manager’s prompt to Documentation Engineer
project_manager_to_documentation_engineer_prompt = (
    "The Test Engineer has completed the testing user stories. Please create user stories for documentation tasks, "
    "including time and effort estimates for each phase. Use the historical waterfall data as a guideline and convert it to Agile using relative sizing, story points, and sprint planning. "
    "Provide a structured report summarizing the user stories, effort estimates, and calculations for the documentation phase. "
    "Include in your report the total number of user stories, pages, and overall effort."

    "Here is the waterfall-based historical data, for reference as you adapt the estimates to Agile:"
    
    "Project 1"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                     134 Pages                 4 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            134 Pages               5 Pages/Hour"
    "       Review Meeting                    134 Pages                7 Pages/Hour"
    "Rework                                   174 defects            6 defects/Hour"

    "Project 2"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                      389 Pages                 6 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            389 Pages                5 Pages/Hour"
    "       Review Meeting                    389 Pages                7 Pages/Hour"
    "Rework                                   532 defects            4 defects/Hour"

    "Project 3"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                      177 Pages                 5 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            177 Pages                5 Pages/Hour"
    "       Review Meeting                    177 Pages                7 Pages/Hour"
    "Rework                                   236 defects            5 defects/Hour"

    "Once completed, report back to me so we can proceed with the next steps in the project plan."
)

project_manager_compiles_backlog = (
    "The outputs from all engineers have been received. I have compiled a complete product backlog with user stories, time, and effort estimates for each phase. "
    "This backlog is now ready for review and further structuring as we move into Scrum planning. "
    "The summary includes total user stories, effort per phase, and initial estimates for each sprint. "
    "Please review the backlog and finalize the sprint structure, defining goals, estimated story points, and sprint duration."
)

scrum_master_structures_plan = (
    "The Project Manager has compiled a comprehensive backlog. I have structured it into an initial sprint plan based on the total user stories and effort estimates. "
    "Drafted an approximate number of sprints, dividing backlog items into sprint-specific goals. "
    "Please review this structured plan, including sprint goals, estimated sprint count, and an outline of the product backlog."
)

product_owner_finalize_backlog = (
    "The Scrum Master has provided an initial sprint plan. I have reviewed and prioritized user stories, adjusting the sprint structure as needed to align with our product goals. "
    "The final sprint breakdown includes prioritized user stories, objectives, and an estimated timeline for each sprint. "
    "The sprint summary is now ready for customer review, including sprint goals, estimated completion time, and total sprints required."
)

product_owner_to_customer_proxy = (
    "The initial planning phase for the AI-powered skin anomaly detection system is complete. Based on your requirements, "
    "we have created a product backlog divided into prioritized sprints with defined user stories and estimated effort for each phase. "
    "The overall project plan is ready for your review. We look forward to moving forward with the first sprint and achieving the key goals you outlined."
)


In [105]:
# Store engineer responses
engineer_outputs = {}

def print_next_speaker(i, queue):
    if i + 1 < len(queue):
        next_speaker = queue[i + 1]['recipient'].name.replace("_", " ")
        print(f"{GREEN}Next Speaker: {next_speaker}{RESET}")
    else:
        print(f"{GREEN}Final Step: Project Manager compiles the final report and sends it to the Scrum Master{RESET}")

def extract_engineer_work(chat_result):
    if chat_result and chat_result.chat_history:
        # Assuming the last entry in chat_history is the engineer's response (output)
        return chat_result.chat_history[-1]['content']
    return 'No response available from the engineer'

def compile_final_project_plan(engineer_outputs):
    final_project_plan = "### Final Project Plan\n\n"
    final_project_plan += "The Initiation and Planning phases are complete. Below is the detailed project plan outlining work completed by each agent:\n\n"
    
    chat_log_index = 1
    
    # Aggregate each agent's structured output
    for agent, details in engineer_outputs.items():
        final_project_plan += (
            f"{BLUE_BOLD}#### Chat Log {chat_log_index}: {agent.replace('_', ' ')} Phase{RESET}\n"
            f"- **Speaker:** {details['speaker'].replace('_', ' ')}\n"
            f"- **Recipient:** {details['recipient'].replace('_', ' ')}\n"
            f"- **Message:**\n{details['input_message']}\n\n"
            f"- **Response:**\n{details['response']}\n\n"
        )
        chat_log_index += 1  # Increment for each phase

    final_project_plan += "### Conclusion:\n"
    final_project_plan += (
        "The project has completed initial stages, including backlog creation with initial user stories estimates. "
        "The project is now ready for further estimating the number of sprints required based on the generated backlog and divide the backlog items for each sprint."
    )
    
    return final_project_plan

def initiate_nested_chats(nested_chat_queue):
    accumulated_work = ""  # Track responses

    # Directly initiate the chain for the Customer Proxy, Product Owner, and Scrum Master
    customer_proxy_output = initiate_single_chat(customer_proxy_agent, product_owner_agent, customer_message)
    product_owner_output = initiate_single_chat(product_owner_agent, scrum_master_agent, product_owner_to_scrum_master_prompt)
    scrum_master_output = initiate_single_chat(scrum_master_agent, project_manager_agent, scrum_master_to_project_manager_prompt)

    # Gather initial work from Customer Proxy, Product Owner, and Scrum Master
    accumulated_work += f"\n\nWork from Customer Proxy:\n{extract_engineer_work(customer_proxy_output)}"
    accumulated_work += f"\n\nWork from Product Owner:\n{extract_engineer_work(product_owner_output)}"
    accumulated_work += f"\n\nWork from Scrum Master:\n{extract_engineer_work(scrum_master_output)}"
    
    # Project Manager loop with each engineer
    project_manager_to_engineers(accumulated_work)

    # Compile the final project plan from all responses
    final_message_to_team = compile_final_project_plan(engineer_outputs)

    # Sequentially send the final project plan up the chain
    send_final_report(final_message_to_team)

def initiate_single_chat(sender, recipient, message):
    print(f"{GREEN}{sender.name.replace('_', ' ')} initiates chat with {recipient.name.replace('_', ' ')}{RESET}")
    output = sender.initiate_chat(agent=sender, recipient=recipient, message=message)
    response = extract_engineer_work(output)
    engineer_outputs[recipient.name] = {
        "speaker": sender.name,
        "recipient": recipient.name,
        "input_message": message,
        "response": response
    }
    return output

def project_manager_to_engineers(accumulated_work):
    engineers = [
        {"agent": requirement_engineer_agent, "message": project_manager_to_requirement_engineer_prompt},
        {"agent": system_engineer_agent, "message": project_manager_to_system_engineer_prompt},
        {"agent": software_engineer_agent, "message": project_manager_to_software_engineer_prompt},
        {"agent": test_engineer_agent, "message": project_manager_to_test_engineer_prompt},
        {"agent": documentation_engineer_agent, "message": project_manager_to_documentation_engineer_prompt},
    ]

    for engineer in engineers:
        message = engineer["message"] + "\n\nHere is the combined work from previous agents:\n" + accumulated_work
        engineer_output = initiate_single_chat(project_manager_agent, engineer["agent"], message)
        
        # Append each engineer's response to accumulated work for subsequent engineers
        accumulated_work += f"\n\nWork from {engineer['agent'].name.replace('_', ' ')}:\n{extract_engineer_work(engineer_output)}"

def send_final_report(final_message_to_team):
    print(f"{GREEN}Project Manager sends the compiled backlog to Scrum Master{RESET}")
    scrum_master_response = project_manager_agent.initiate_chat(
        agent=project_manager_agent,
        recipient=scrum_master_agent,
        message=project_manager_compiles_backlog + "\n\n" + final_message_to_team
    )

    print(f"{GREEN}Scrum Master structures the backlog for sprint planning and sends it to Product Owner{RESET}")
    product_owner_response = scrum_master_agent.initiate_chat(
        agent=scrum_master_agent,
        recipient=product_owner_agent,
        message=scrum_master_structures_plan + "\n\n" + extract_engineer_work(scrum_master_response)
    )

    print(f"{GREEN}Product Owner finalizes the sprint plan and sends it to Customer Proxy{RESET}")
    product_owner_response = product_owner_agent.initiate_chat(
        agent=product_owner_agent,
        recipient=customer_proxy_agent,
        message=product_owner_finalize_backlog + "\n\n" + extract_engineer_work(product_owner_response)
    )

    print(f"{GREEN}Product Owner provides the final overview to Customer Proxy{RESET}")
    product_owner_agent.initiate_chat(
        agent=product_owner_agent,
        recipient=customer_proxy_agent,
        message=product_owner_to_customer_proxy + extract_engineer_work(product_owner_response)
    )

# Define the nested chat queue for initial steps only
nested_chat_queue = [
    {"sender": customer_proxy_agent, "recipient": product_owner_agent, "message": customer_message},
    {"sender": product_owner_agent, "recipient": scrum_master_agent, "message": product_owner_to_scrum_master_prompt},
    {"sender": scrum_master_agent, "recipient": project_manager_agent, "message": scrum_master_to_project_manager_prompt},
]

# Start the adjusted nested chat sequence
initiate_nested_chats(nested_chat_queue=nested_chat_queue)


Customer Proxy initiates chat with Product Owner
Customer_Proxy (to Product_Owner):

I want to create an AI-powered computer vision system that detects anomalies in the skin, such as cancer and psoriasis. The system should process high-resolution images, provide real-time diagnostics, and support doctors with detailed reports. The priorities are accuracy, speed, and usability in a healthcare setting.

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Product_Owner (to Customer_Proxy):

### Product Vision
Develop an AI-powered computer vision system that utilizes advanced machine learning algorithms to accurately detect skin anomalies such as cancer and psoriasis from high-resolution images. The system will provide real-time diagnostic support, generating detailed reports for healthcare professionals to enhance their diagnostic capabilities, ultimately leading to improved patient outcomes.

#